# Setting up scraping environment

### Install selenium

In [1]:
# install selenium
# !pip install selenium
import time
import pandas as pd 
import numpy as np
import os

### download and install firefox gecko driver from following link
https://github.com/mozilla/geckodriver/releases/

Initialise selenium browser

In [2]:
# from selenium import webdriver
# from selenium.webdriver.firefox.service import Service as FirefoxService
# from webdriver_manager.firefox import GeckoDriverManager

# browser = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

In [4]:
from selenium import webdriver
binary = '/usr/bin/firefox'
options = webdriver.FirefoxOptions()
# # options.binary = binary
# options.add_argument('start-maximized')
# # options.add_argument('--headless')
# browser = webdriver.Firefox(firefox_options=options, executable_path='/usr/bin/geckodriver')
browser = webdriver.Firefox(options=options)

# # leaderboardURL = "https://www.kaggle.com/competitions/students-drop-out-prediction/leaderboard"
# # browser.get(leaderboardURL)

In [5]:
from selenium.webdriver.common.by import By

In [6]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By

# from selenium import webdriver
# from selenium.webdriver.firefox.service import Service as FirefoxService
# from webdriver_manager.firefox import GeckoDriverManager

# import pandas as pd
# import numpy as np
# import time

# browser = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

## Login with username password
This avoids signing in from google account

In [7]:
# url to the submissions page
browser.get("https://www.kaggle.com/account/login?phase=emailSignIn&returnUrl=%2F")

# Find the email input field and enter the email
username = ""
password = ""


time.sleep(10)
email_input = browser.find_element (By.NAME, "email")
email_input.clear()  # Clear any existing value
email_input.send_keys(username)

# Print a success message
print("Email entered successfully.")

# Find the email input field and enter the email
password_input = browser.find_element(By.NAME, "password")
password_input.clear()  # Clear any existing value
password_input.send_keys(password)


# Print a success message
print("password entered successfully.")

# Find the 'Sign In' button and click it
sign_in_button = browser.find_element(By.XPATH, "//button[contains(., 'Sign In')]")
sign_in_button.click()

# Print a success message
print("Successfully clicked the 'Sign In' button.")


Email entered successfully.
password entered successfully.
Successfully clicked the 'Sign In' button.


In [8]:
# url to the submissions page
time.sleep(5)
browser.get("https://www.kaggle.com/competitions/crime-cast-forecasting-crime-categories/host/teams-submissions")
time.sleep(5)

In [14]:
# get number of students
total_team_count = 0
import re
m = re.search(r'(\d+) teams . (\d+) submissions',browser.page_source)
total_team_count = int(m.groups(1)[0])

In [16]:
total_team_count

936

In [17]:
# class of each subrow ="MuiDataGrid-row"

In [18]:
# parentstudents = browser.find_elements_by_xpath("//div[@class='MuiDataGrid-virtualScrollerRenderZone*']")
# parentstudents = browser.find_elements_by_xpath("//aside//div[starts-with(@class, 'MuiDataGrid-virtualScrollerContent')]")
parentstudents = browser.find_elements_by_xpath("(//div[@class='MuiDataGrid-virtualScrollerContent css-0'])[1]")


# parentstudents = browser.find_elements_by_xpath("//div[@class='MuiDataGrid-virtualScroller']")
# parentstudents = browser.find_elements_by_xpath("//div[@class='MuiDataGrid-virtualScrollerContent']//div[@class='MuiDataGrid-row' AND @role='row']")
# parentstudents = browser.find_elements_by_xpath("//div[@class='MuiDataGrid-virtualScroller']")
# parentstudents = browser.find_elements_by_xpath("//main//div[@role='row']")


In [19]:
len(parentstudents)

1

In [20]:
'''
Turns the page of the leaderboard or
shows next set of entries in the leaderboard
'''

def goto_next_page():
    # find the button to get/show next entris in the leaderboard
    next_button = browser.find_elements_by_xpath("//button[@title='Go to next page']")
    
    # check if already at the last page, if the last page is reached, 
    # the button will be disabled, hence it will have an attribute called
    # 'disabled'
    if not next_button[0].get_attribute('disabled'):
        browser.execute_script("arguments[0].click();", next_button[0])
        
        # there are no pages left to turn
        return True
    
    # there are more pages left
    return False

In [21]:
# test page turning
# goto_next_page()

In [22]:
close_button = browser.find_elements_by_xpath("//button[@aria-label='close']")
len(close_button)

0

In [23]:
def close_subrows():
    # find the close button of the drawer windos
    close_button = browser.find_elements_by_xpath("//button[@aria-label='close']")

    # click on the close button
    browser.execute_script("arguments[0].click();", close_button[0])

In [24]:
# test closing of side drawer
# close_subrows()

In [25]:
'''
Return the submission' data for the student
whose drawer window/pop is opened
'''
def get_student_submissions(printOnly=False):
    # in xpath, the counting begins from 1, hence select the second element
#     old
#     subrows = browser.find_elements_by_xpath("(//div[@class='MuiDataGrid-virtualScrollerContent css-0'])[2]")
    subrows = browser.find_elements_by_xpath("(//div[@class='MuiDataGrid-virtualScrollerContent css-0'])[1]")
    
    assert len(subrows) >0, "Could not access drawer window"
    
#     old
#     subrows = subrows[0].find_elements_by_xpath(".//div[@role='cell']")
    subrows = subrows[0].find_elements_by_xpath(".//div[@data-id]")
    
    # a temporary dataframe to hold submissions' info of current student
    tdf = pd.DataFrame(columns = ['id','name','details','date'])    
    assert len(subrows)>0,"No subrows selected"
    
    # number of entries/submission by the student
    print('\t\t\t\tSubrows detected',len(subrows))
    for i_row in subrows:
                
        submission_id = i_row.find_elements_by_xpath(".//div[@data-field='id']")[0]
        submission_name = i_row.find_elements_by_xpath(".//div[@data-field='name']")[0]
        submission_details = i_row.find_elements_by_xpath(".//div[@data-field='details']")[0]
        submission_date = i_row.find_elements_by_xpath(".//div[@data-field='date']")[0]

        if printOnly:
            print(submission_id.get_attribute('innerText'), 
                  submission_name.get_attribute('innerText'), 
                  submission_date.get_attribute('innerText'), 
                  submission_details.get_attribute('innerText'), sep=" ")
        else:
            # append the submission to temporary dataframe
            tdf.loc[len(tdf.index)]= [submission_id.get_attribute('innerText'), 
                    submission_name.get_attribute('innerText'),                      
                    submission_details.get_attribute('innerText'),
                    submission_date.get_attribute('innerText')]
            
    close_subrows()
    time.sleep(2)
    
    if not printOnly:
        return tdf
    

    

In [26]:
page_counter = 1
student_counter = 1 

output_file_path = r"C:\Users\manoj\Downloads\MLPT22024_submissions.csv"
# output_file_path = r"C:\Users\soni\Downloads\MLPT12024_submissions.csv"

page_dir = r"C:\Users\manoj\Downloads"

# the dataframe that will collect submission of each student
submissions_df = pd.DataFrame(columns = ['id','name','details','date'])

In [33]:
# restart the process from intermediate state
page_counter = 14
student_counter = 651 
submissions_df=pd.read_csv(output_file_path)

In [34]:
# before running following loop,
# make sure that the first page of the leader board
# is on screen, otherwise the results will be incomplete
while True:
    
    print('\t\tstarted page #',page_counter)
    # Create a dataframe for current page 
    globals()[f'page_df{page_counter}'] = pd.DataFrame(columns = ['id','name','details','date'])
    # in xpath, the counting begins from 1, hence select the first element
    leaderboard_rows = browser.find_elements_by_xpath("(//div[@class='MuiDataGrid-virtualScrollerContent css-0'])[1]")

    # find kaggle diplay name fields to click and expand 
    # the student's submission
    leaderboard_rows = leaderboard_rows[0].find_elements_by_xpath(".//div[@data-field='name']")
    for i_row in leaderboard_rows:
        
        print('\t\t\tStarted student #',student_counter)
    #     i_name_field = i_row.find_elements_by_xpath(".//div[@data-field='rank']")[0]
    #     print(i_rank_field)
        # expand a student's submission
        browser.execute_script("arguments[0].click();", i_row)

        # wait for the submissions to load
        time.sleep(5)
        
        # collect submmissions data
        a_subrow = get_student_submissions()
        
        # append to current page dataframe
        globals()[f'page_df{page_counter}'] = pd.concat([globals()[f'page_df{page_counter}'], a_subrow], ignore_index=True)

        # append to the main dataframe
#         submissions_df= pd.concat([submissions_df,a_subrow],ignore_index=True)
        
        # wait for some before next student's
        # records are expanded
        time.sleep(3)
        
        # close_subrows() is not working as of now,
        # hence commenting out
        # close_subrows()
    
        print('\t\t\tCompleted student #',student_counter)
        student_counter+=1
    
    # turn page and see if reached the last page
    if not goto_next_page():
        break
    else:
        
        globals()[f'page_df{page_counter}'].to_csv(os.path.join(page_dir, f"page_df{page_counter}.csv"), index=False)
        # append to the main dataframe
        submissions_df = pd.concat([submissions_df, globals()[f'page_df{page_counter}']], ignore_index=True)
        
        # write the dataframe for safety after every page
        submissions_df.to_csv(output_file_path, index=False)
        
        # allow some time for the next page to load
        time.sleep(3)

        page_counter+=1
        
# write the dataframe with last page submissions also
submissions_df.to_csv(output_file_path,index=False)

		started page # 14
			Started student # 651
				Subrows detected 2
			Completed student # 651
			Started student # 652
				Subrows detected 4
			Completed student # 652
			Started student # 653
				Subrows detected 7
			Completed student # 653
			Started student # 654
				Subrows detected 5
			Completed student # 654
			Started student # 655
				Subrows detected 11
			Completed student # 655
			Started student # 656
				Subrows detected 21
			Completed student # 656
			Started student # 657
				Subrows detected 3
			Completed student # 657
			Started student # 658
				Subrows detected 7
			Completed student # 658
			Started student # 659
				Subrows detected 5
			Completed student # 659
			Started student # 660
				Subrows detected 7
			Completed student # 660
			Started student # 661
				Subrows detected 3
			Completed student # 661
			Started student # 662
				Subrows detected 4
			Completed student # 662
			Started student # 663
				Subrows detected 19
			Completed student # 663
		

			Completed student # 759
			Started student # 760
				Subrows detected 1
			Completed student # 760
			Started student # 761
				Subrows detected 2
			Completed student # 761
			Started student # 762
				Subrows detected 1
			Completed student # 762
			Started student # 763
				Subrows detected 1
			Completed student # 763
			Started student # 764
				Subrows detected 1
			Completed student # 764
			Started student # 765
				Subrows detected 1
			Completed student # 765
			Started student # 766
				Subrows detected 1
			Completed student # 766
			Started student # 767
				Subrows detected 2
			Completed student # 767
			Started student # 768
				Subrows detected 10
			Completed student # 768
			Started student # 769
				Subrows detected 4
			Completed student # 769
			Started student # 770
				Subrows detected 1
			Completed student # 770
			Started student # 771
				Subrows detected 1
			Completed student # 771
			Started student # 772
				Subrows detected 5
			Completed student # 7

				Subrows detected 1
			Completed student # 868
			Started student # 869
				Subrows detected 4
			Completed student # 869
			Started student # 870
				Subrows detected 3
			Completed student # 870
			Started student # 871
				Subrows detected 1
			Completed student # 871
			Started student # 872
				Subrows detected 1
			Completed student # 872
			Started student # 873
				Subrows detected 11
			Completed student # 873
			Started student # 874
				Subrows detected 2
			Completed student # 874
			Started student # 875
				Subrows detected 2
			Completed student # 875
			Started student # 876
				Subrows detected 1
			Completed student # 876
			Started student # 877
				Subrows detected 2
			Completed student # 877
			Started student # 878
				Subrows detected 3
			Completed student # 878
			Started student # 879
				Subrows detected 2
			Completed student # 879
			Started student # 880
				Subrows detected 4
			Completed student # 880
			Started student # 881
				Subrows detected 1
	

In [35]:
# write the dataframe with last page submissions also
submissions_df.to_csv(output_file_path,index=False)

In [36]:
submissions_df.tail()

,id,name,details,date
9734,38836889,submission.csv,Notebook 21f3003005-notebook-t22024 | Version 13,"23/6/2024, 10:34:02 am"
9735,38825773,submission.csv,Notebook 21f3003005-notebook-t22024 | Version 12,"22/6/2024, 6:02:39 pm"
9736,38766238,submission.csv,Notebook 21f3003005-notebook-t22024 | Version 4,"18/6/2024, 8:17:35 pm"
9737,38744642,submission.csv,Notebook 21f3003005-notebook-t22024 | Version 2,"17/6/2024, 10:25:50 am"
9738,38743705,submission.csv,,"17/6/2024, 8:35:35 am"


In [37]:
submissions_df.shape

(9739, 4)

In [78]:
# browser.close()